In [1]:
import xarray as xr
import numpy as np
import scipy as sc
import sklearn as skl
import skimage as ski
from scipy.linalg import norm
from scipy.spatial.distance import euclidean, jensenshannon, correlation
from scipy.stats import wasserstein_distance, ecdf
import skgstat as skg
import pysteps
from pysteps.verification.spatialscores import fss


Pysteps configuration file found at: /work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/pysteps/pystepsrc



In [ ]:
datapath='/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/'
models=['graphcast','graphcast-oper','pangu','pangu-oper','ifs']
references=['era','ifs_init']

In [8]:
era_ref=xr.open_dataset(datapath+'USA_tor_2020_era.nc')
nn=0
model=models[nn]
model_set=xr.open_dataset(datapath+'USA_tor_2020_graphcast_2020041100.nc')

In [27]:
era_ref.cape

<xarray.DataArray 'cape' (time: 8, latitude: 121, longitude: 201)>
array([[[        nan,         nan, ...,         nan,         nan],
        [        nan,         nan, ...,         nan,         nan],
        ...,
        [        nan,         nan, ...,  407.019269,  383.813132],
        [        nan,         nan, ...,  345.00178 ,  332.110236]],

       [[        nan,         nan, ...,         nan,         nan],
        [        nan,         nan, ...,         nan,         nan],
        ...,
        [        nan,         nan, ...,  339.726223,  349.714134],
        [        nan,         nan, ...,  248.295757,  294.51073 ]],

       ...,

       [[        nan,         nan, ...,         nan,         nan],
        [        nan,         nan, ...,         nan,         nan],
        ...,
        [        nan,         nan, ...,  740.834072,  755.20794 ],
        [        nan,         nan, ...,  708.881672,  717.002704]],

       [[        nan,         nan, ...,         nan,         nan],
        [        nan,         nan, ...,         nan,         nan],
        ...,
        [        nan,         nan, ..., 1061.812497,  949.543478],
        [        nan,         nan, ...,  892.38871 ,  788.538698]]])
Coordinates:
  * latitude            (latitude) float32 50.0 49.75 49.5 ... 20.5 20.25 20.0
    level               int64 ...
  * longitude           (longitude) float32 250.0 250.2 250.5 ... 299.8 300.0
  * time                (time) datetime64[ns] 2020-04-12 ... 2020-04-13T18:00:00
    mcape_mcin_lcl_lfc  <U5 ...
Attributes:
    description:  mcape ; mcin ; lcl ; lfc
    units:        J kg-1 ; J kg-1 ; m ; m
    MemoryOrder:

In [31]:

lat, lon =np.meshgrid(era_ref.latitude,era_ref.longitude)

lat=lat.flatten(); lon=lon.flatten(); cape=era_ref.cape.fillna(0).values[0,:,:].flatten()
print(lat.shape)
print(cape.shape)
coords=np.zeros([len(lat),2])
coords[:,0]=lon; coords[:,1]=lat
print(coords.shape)
V = skg.Variogram(coords, cape)
# #determine variogram range to obtain distance, where data is uncorrelated
[range, sill, nugget] = V.parameters

(24321,)
(24321,)
(24321, 2)


In [38]:
print(range, sill, nugget)
print(model_set.cape.squeeze().shape)
print(era_ref.cape.shape)

54.92576955918405 630108.0001478283 0
(8, 121, 201)
(8, 121, 201)


In [52]:
fss_eval=fss(model_set.cape.values[0,0,:,:], np.flipud(era_ref.cape.values[0,:,:]), 300, scale=range)
print(fss_eval)

0.992656738399136


In [45]:
print(fss_eval)
print(model_set.lat,era_ref.latitude)

0.992656738399136
<xarray.DataArray 'lat' (lat: 121)>
array([20.  , 20.25, 20.5 , 20.75, 21.  , 21.25, 21.5 , 21.75, 22.  , 22.25,
       22.5 , 22.75, 23.  , 23.25, 23.5 , 23.75, 24.  , 24.25, 24.5 , 24.75,
       25.  , 25.25, 25.5 , 25.75, 26.  , 26.25, 26.5 , 26.75, 27.  , 27.25,
       27.5 , 27.75, 28.  , 28.25, 28.5 , 28.75, 29.  , 29.25, 29.5 , 29.75,
       30.  , 30.25, 30.5 , 30.75, 31.  , 31.25, 31.5 , 31.75, 32.  , 32.25,
       32.5 , 32.75, 33.  , 33.25, 33.5 , 33.75, 34.  , 34.25, 34.5 , 34.75,
       35.  , 35.25, 35.5 , 35.75, 36.  , 36.25, 36.5 , 36.75, 37.  , 37.25,
       37.5 , 37.75, 38.  , 38.25, 38.5 , 38.75, 39.  , 39.25, 39.5 , 39.75,
       40.  , 40.25, 40.5 , 40.75, 41.  , 41.25, 41.5 , 41.75, 42.  , 42.25,
       42.5 , 42.75, 43.  , 43.25, 43.5 , 43.75, 44.  , 44.25, 44.5 , 44.75,
       45.  , 45.25, 45.5 , 45.75, 46.  , 46.25, 46.5 , 46.75, 47.  , 47.25,
       47.5 , 47.75, 48.  , 48.25, 48.5 , 48.75, 49.  , 49.25, 49.5 , 49.75,
       50.  ], dtype=f

In [5]:
## https://gist.github.com/larsmans/3116927
_SQRT2 = np.sqrt(2)     # sqrt(2) with default precision np.float64


def hellinger1(p, q):
    return norm(np.sqrt(p) - np.sqrt(q)) / _SQRT2


def hellinger2(p, q):
    return euclidean(np.sqrt(p), np.sqrt(q)) / _SQRT2


def hellinger3(p, q):
    return np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q)) ** 2)) / _SQRT2

In [7]:
ecdf(sample)

pdf = derivative(ecdf)

jensenshannon(p, q, base=None, *, axis=0, keepdims=False)
# probability distribution -> requires ecdf and pdf

wasserstein_distance(u_values, v_values, u_weights=None, v_weights=None)
# probability distribution -> requires ecdf and pdf

correlation(u, v, w=None, centered=True)
#correlation distance

V = skg.Variogram(coords, values)
#determine variogram range to obtain distance, where data is uncorrelated
[range, sill, nugget] = V.parameters


fss(X_f, X_o, thr, scale=range)


SyntaxError: iterable argument unpacking follows keyword argument unpacking (1837290051.py, line 5)